In [100]:
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
import missingno as msno
import time 
import sys
import os
import csv
from xgboost import XGBClassifier
import requests
import re
from sklearn.model_selection import cross_val_predict

In [101]:
df=pd.read_csv("/home/apprenant/Documents/DATA/loan_project/SBAnational.csv")

/tmp/ipykernel_6363/2889706020.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/home/apprenant/Documents/DATA/loan_project/SBAnational.csv")


In [102]:
df['ApprovalFY'] = df['ApprovalFY'].replace('A', '', regex=True).astype(int)

In [103]:
columns_to_transform_to_int = ["DisbursementGross","BalanceGross","ChgOffPrinGr","GrAppv","SBA_Appv"]


for col in columns_to_transform_to_int:
    df[col] = df[col].str.replace("$", "")
    df[col] = df[col].str.replace(",", "")
    df[col] = df[col].astype(float)
    

df['Term'] = df['Term'].astype(int)

/tmp/ipykernel_6363/2501809618.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace("$", "")


In [104]:
df['LowDoc'] = df['LowDoc'].replace({'0': 'N'})
df['LowDoc'].value_counts()

N    784313
Y    110335
C       758
S       603
A       497
R        75
1         1
Name: LowDoc, dtype: int64

In [105]:
df['RevLineCr'] = df['RevLineCr'].replace({'0': 'N', 'T':'Y'})
df['RevLineCr'].value_counts()

N    677890
Y    216681
1        23
R        14
`        11
2         6
C         2
,         1
3         1
7         1
A         1
5         1
.         1
4         1
-         1
Q         1
Name: RevLineCr, dtype: int64

In [106]:
df = df[df['RevLineCr'].isin(['N','Y'])]
df['RevLineCr'].value_counts()

N    677890
Y    216681
Name: RevLineCr, dtype: int64

In [107]:
dictionnaire =  {"11":"Agriculture, forestry, fishing and hunting","72":"Accommodation and food services","21":"Mining, quarrying, and oil and gas extraction","22":
"Utilities","23":"Construction","31":"Manufacturing","32":"Manufacturing","33":"Manufacturing","42":"Wholesale trade","44":"Retail trade","45":"Retail trade","48":" Transportation and warehousing", "49":"Transportation and warehousing", "51":"Information","52":"Finance and insurance", "53":"Real estate and rental and leasing","54":"Professional, scientific, and technical services","55":"Management of companies and enterprises","56":"Administrative and support and waste management and remediation services","61":"Educational services","62":"Health care and social assistance","71":"Arts, entertainment, and recreation","81":"Other services (except public administration)", "92": "Public administration", "0" : "Other"}

df['NAICS'] = df['NAICS'].astype(str).str[:2]

df["Categorie_NAICS"] = df["NAICS"].map(dictionnaire)

# df.Categorie.value_counts(dropna=False)

df["Categorie_NAICS"].value_counts(dropna=False)

Other                                                                       199739
Retail trade                                                                126783
Other services (except public administration)                                72381
Professional, scientific, and technical services                             67948
Manufacturing                                                                67728
Accommodation and food services                                              67474
Construction                                                                 66408
Health care and social assistance                                            55108
Wholesale trade                                                              48555
Administrative and support and waste management and remediation services     32609
 Transportation and warehousing                                              20287
Arts, entertainment, and recreation                                          14597
Real

In [108]:
cols_to_drop = ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'RetainedJob','ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'SBA_Appv', 'ChgOffPrinGr']
df_cleaned = df.copy()
df_cleaned.drop(columns=cols_to_drop, inplace=True)

In [109]:
df_cleaned.dropna(subset=['MIS_Status'], inplace=True)

In [110]:
y = df_cleaned.MIS_Status
X = df_cleaned.drop(columns=['MIS_Status'])

In [123]:
X_train.head(1)

,State,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv,Categorie_NAICS
209094,IL,31,2007,12,1,2.0,0,0,1,Y,N,350000.0,Manufacturing


Les hyperparamètres les plus importants pour XGBoost Classifier sont :

n_estimators : le nombre d'arbres de décision dans le modèle
learning_rate : le taux d'apprentissage utilisé pour mettre à jour les poids des modèles
max_depth : la profondeur maximale de chaque arbre de décision
subsample : la proportion d'échantillons utilisés pour construire chaque arbre
colsample_bytree : la proportion de colonnes utilisées pour construire chaque arbre
gamma : un paramètre qui régule la complexité de chaque arbre.
reg_alpha : la régularisation L1 sur les poids
reg_lambda : la régularisation L2 sur les poids.
scale_pos_weight : pour gérer les déséquilibres de classe
objective : choisir l'objectif de la régression, logistique pour la classification binaire, softmax pour la classification multi-classe.

In [111]:
from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.


from sklearn.impute import SimpleImputer
import numpy as np

def make_pipeline_to_ML(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.9, random_state=42)
    numerical_features = make_column_selector(dtype_include=np.number)
    categorical_features = make_column_selector(dtype_exclude= np.number)
    numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler(with_mean=False))
    categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown='ignore'))
    preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                    (categorical_pipeline, categorical_features)
                                    )
    return preprocessor, X_train, X_test, y_train, y_test


# Create the pipeline
preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X,y)

In [112]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 803323 entries, 209094 to 122762
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   State            803314 non-null  object 
 1   NAICS            803323 non-null  object 
 2   ApprovalFY       803323 non-null  int64  
 3   Term             803323 non-null  int64  
 4   NoEmp            803323 non-null  int64  
 5   NewExist         803211 non-null  float64
 6   CreateJob        803323 non-null  int64  
 7   FranchiseCode    803323 non-null  int64  
 8   UrbanRural       803323 non-null  int64  
 9   RevLineCr        803323 non-null  object 
 10  LowDoc           801038 non-null  object 
 11  GrAppv           803323 non-null  float64
 12  Categorie_NAICS  803323 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 85.8+ MB


In [113]:
y_train

209094     P I F
249452     P I F
869307     P I F
91777      P I F
807410     P I F
           ...  
260353     P I F
367137     P I F
132769     P I F
673408     P I F
122762    CHGOFF
Name: MIS_Status, Length: 803323, dtype: object

In [114]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoder = le.fit_transform(y_train)
y_test_encoder = le.fit_transform(y_test)

In [115]:
decoded_data = le.inverse_transform(y_train_encoder)
decoded_data

array(['P I F', 'P I F', 'P I F', ..., 'P I F', 'P I F', 'CHGOFF'],
      dtype=object)

In [116]:
estimator = make_pipeline(preprocessor, XGBClassifier(
    objective= 'binary:logistic',    tree_method = 'hist', random_state =42, n_jobs=-1
))
param_grid = {'xgbclassifier__max_depth': [10],
            'xgbclassifier__n_estimators': [200],
              'xgbclassifier__learning_rate': [ 0.2],
            
             'xgbclassifier__colsample_bytree' : [ 0.7],
            'xgbclassifier__scale_pos_weight' : [ 1]
            }

In [117]:
# grid_search2 = make_pipeline(preprocessor, GridSearchCV(
#     estimator=estimator,
#     param_grid=param_grid,


 
# ))

In [118]:
grid_search2 = GridSearchCV(estimator,param_grid,cv=5 , verbose=6)

In [121]:
grid_bost  = grid_search2.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=200, xgbclassifier__scale_pos_weight=1;, score=nan total time=   7.2s
[CV 2/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=200, xgbclassifier__scale_pos_weight=1;, score=nan total time=   7.3s
[CV 3/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=200, xgbclassifier__scale_pos_weight=1;, score=nan total time=   7.6s
[CV 4/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=200, xgbclassifier__scale_pos_weight=1;, score=nan total time=   7.3s
[CV 5/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__learning_rate=0.2, xgbclassifier

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/apprenant/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/apprenant/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/sklearn/pipeline.py", line 406, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/apprenant/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/home/apprenant/miniconda3/envs/data_analyse_env/lib/python3.10/site-packages/xgboost/sklearn.py", 

In [ ]:
print("les meilleurs paramètres sont :",grid_bost.best_params_)
print("le meilleurs score sont :",grid_bost.best_score_)




les meilleurs paramètres sont : {'xgbclassifier__colsample_bytree': 0.7, 'xgbclassifier__learning_rate': 0.2, 'xgbclassifier__max_depth': 10, 'xgbclassifier__n_estimators': 200, 'xgbclassifier__scale_pos_weight': 1}
le meilleurs score sont : 0.9490989314566749


In [ ]:


#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

y_pred = grid_search2.predict(X_train)
# Model Accuracy, how often is the classifier correct?
print ("Accuracy:", {metrics.accuracy_score(y_train, y_pred)})
print ("f1_SCORE_WEIGHTED:", {metrics.f1_score(y_train, y_pred, average='weighted')})
print ("f1_SCORE_MACRO:", {metrics.f1_score(y_train, y_pred, average='macro')})

Accuracy: {0.9616903785899321}
f1_SCORE_WEIGHTED: {0.9614010703305332}
f1_SCORE_MACRO: {0.9331069402909564}


In [ ]:
y_pred2 = grid_search2.predict(X_test)
print ("Accuracy:", {metrics.accuracy_score(y_test, y_pred2)})
print ("f1_SCORE_WEIGHTED:", {metrics.f1_score(y_test, y_pred2, average='weighted')})
print ("f1_SCORE_MACRO:", {metrics.f1_score(y_test, y_pred2, average='macro')})

Accuracy: {0.9497641694394963}
f1_SCORE_WEIGHTED: {0.9493551081729162}
f1_SCORE_MACRO: {0.9118852256039505}


In [ ]:
len(X_test) 

89259

In [ ]:
len(y_test)

89259

In [ ]:
print(metrics.classification_report(y_train, y_pred))


              precision    recall  f1-score   support

           0       0.91      0.87      0.89    141760
           1       0.97      0.98      0.98    661563

    accuracy                           0.96    803323
   macro avg       0.94      0.93      0.93    803323
weighted avg       0.96      0.96      0.96    803323



In [ ]:
print(metrics.classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85     15684
           1       0.97      0.97      0.97     73575

    accuracy                           0.95     89259
   macro avg       0.92      0.91      0.91     89259
weighted avg       0.95      0.95      0.95     89259



In [ ]:
xgb_clf = make_pipeline(preprocessor, XGBClassifier(n_estimators=500, 
                            learning_rate=0.1, 
                            max_depth=5, 
                            subsample=0.8, 
                            colsample_bytree=0.8, 
                            gamma=0, 
                            reg_alpha=0, 
                            reg_lambda=1,
                            scale_pos_weight=1,
                            tree_method="hist",
                            objective='binary:logistic'))

# Entraîner le modèle sur les données d'entraînement
xgb_clf.fit(X_train, y_train)

# Prédire les données de test
y_pred3 = xgb_clf.predict(X_test)

In [ ]:

print ("Accuracy:", {metrics.accuracy_score(y_test, y_pred3)})
print ("f1_SCORE_WEIGHTED:", {metrics.f1_score(y_test, y_pred3, average='weighted')})
print ("f1_SCORE_MACRO:", {metrics.f1_score(y_test, y_pred3, average='macro')})

Accuracy: {0.94752349903091}
f1_SCORE_WEIGHTED: {0.9469065243927655}
f1_SCORE_MACRO: {0.9073086543519269}


In [ ]:
print(metrics.classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.87      0.82      0.85     15684
           1       0.96      0.97      0.97     73575

    accuracy                           0.95     89259
   macro avg       0.92      0.90      0.91     89259
weighted avg       0.95      0.95      0.95     89259



In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Définir les valeurs des hyperparamètres à tester
param_grid = {'xgbclassifier__n_estimators': [100],
              'xgbclassifier__learning_rate': [0.2],
              'xgbclassifier__max_depth': [10],
              'xgbclassifier__subsample': [ 0.8],
              'xgbclassifier__colsample_bytree': [ 0.7],
              'xgbclassifier__gamma': [0.5],
             'xgbclassifier__reg_alpha': [ 1],
             'xgbclassifier__reg_lambda': [ 1],
          'xgbclassifier__scale_pos_weight': [1],
             }

# Initialiser le modèle
xgb_clf = make_pipeline(preprocessor,xgb.XGBClassifier(objective= 'binary:logistic',    tree_method = 'hist'))

# Initialiser la recherche d'hyperparamètres
grid_search = GridSearchCV(xgb_clf, param_grid, cv=5, n_jobs=-1,verbose=4)

# Entraîner le modèle en utilisant la recherche d'hyperparamètres
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres : ", grid_search.best_params_)



Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__gamma=0.5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=100, xgbclassifier__reg_alpha=1, xgbclassifier__reg_lambda=1, xgbclassifier__scale_pos_weight=1, xgbclassifier__subsample=0.8;, score=0.948 total time= 1.4min
[CV 5/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__gamma=0.5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=100, xgbclassifier__reg_alpha=1, xgbclassifier__reg_lambda=1, xgbclassifier__scale_pos_weight=1, xgbclassifier__subsample=0.8;, score=0.949 total time= 1.4min
[CV 2/5] END xgbclassifier__colsample_bytree=0.7, xgbclassifier__gamma=0.5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=10, xgbclassifier__n_estimators=100, xgbclassifier__reg_alpha=1, xgbclassifier__reg_lambda=1, xgbclassifier__scale_pos_weight=1, xgbclassifier__subsample=

In [ ]:
print("Meilleurs hyperparamètres : ", grid_search.best_score_)

Meilleurs hyperparamètres :  0.9486034893154385


In [ ]:
y_pred4 = grid_search.predict(X_test)
print ("Accuracy:", {metrics.accuracy_score(y_test, y_pred4)})
print ("f1_SCORE_WEIGHTED:", {metrics.f1_score(y_test, y_pred4, average='weighted')})
print ("f1_SCORE_MACRO:", {metrics.f1_score(y_test, y_pred4, average='macro')})

Accuracy: {0.94947288228638}
f1_SCORE_WEIGHTED: {0.9489794507940464}
f1_SCORE_MACRO: {0.9110945137612918}
